## Bartosz Sroka

In [ ]:
# Import opowiednich bibliotek
import pandas as pd
import numpy as np
import os

In [23]:
# Wybór odpowiedniego zadania
def choose_data(surname):
    list_of_data = ['weather.txt', 'tb.csv', 'billboard.csv', 'drinks.csv', 'earthquake_data.csv']
    print("Wybrane zadanie to:\t " + list_of_data[len(surname)%5])
choose_data("sroka")

Wybrane zadanie to:	 weather.txt


In [21]:
# # Wczytanie danych 
# if not os.path.exists("weather.txt"):
#     !wget https://raw.githubusercontent.com/barteks019/AiBD_BartoszSroka_1b/blob/main/Lab%202%20-%2013.10.2021/OriginalData/weather.txt

In [122]:
weather = open('..\OriginalData\weather.txt', 'r')

# Funkcja przyjmująca dane w formacie .txt, konwertująca dane do postaci czytelnej przez pakiet pandas
def convert_data(data, year, number_of_months):
    converted_data = []
    while True:
        line = data.readline()
        # Wykonanie konwersji "-" na " -" pozwala na odczytanie wartości, które zostały ze sobą połączone przez zbyt dużą długość wartości (-9999)
        splited_line = [x for x in line.replace('-', ' -').split(' ')[:-1] if x]
        if splited_line == []:
            break
        # Rozdzielenie pierwszej wartości z linii na wartość id_, year, month, type.
        splited_line = [splited_line[0][:11], int(splited_line[0][11:15]), int(splited_line[0][-6:-4]), splited_line[0][-4:]] + splited_line[1:]
        # wybranie n pierwszych miesięcy w wybranym roku.
        if splited_line[1] == year and splited_line[2] <= number_of_months:
            # Przefiltrowanie danych z validatorów S i I, wszystkie wartości w roku 2010 są obarczone takim samym validatorem daltego się go pozbywam.
            filtered_data_from_validators = [x for x in splited_line[4:] if x != "I" if x != "S"]
            # Zmiana formatu danych wejściowych (str2int)
            change_format_of_temperature_to_int = [int(x) for x in filtered_data_from_validators]
            # Usuwanie nieprawidłowych danych:
                # Przy rzeczywistych danych pobieranych na ziemi w skali Kelwina nie mamy doczynienia z temperaturą równą 0 [K] (zera bezwględnego)
                # Wartości w skali Kelwina są wyłącznie dodatnie daltego temperatura -9999 [K] nie ma prawa istnieć.
            filtered_data_from_invalid_numbers = splited_line[:4] + [x if x > 0 else np.nan for x in change_format_of_temperature_to_int]
            converted_data.append(filtered_data_from_invalid_numbers)
    return converted_data

[print(x) for x in convert_data(weather, 2010, 5)];

['MX000017004', 2010, 1, 'TMAX', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 278, nan]
['MX000017004', 2010, 1, 'TMIN', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 145, nan]
['MX000017004', 2010, 1, 'PRCP', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
['MX000017004', 2010, 2, 'TMAX', nan, 273, 241, nan, nan, nan, nan, nan, nan, nan, 297, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 299, nan, nan, nan, nan, nan, nan, nan, nan]
['MX000017004', 2010, 2, 'TMIN', nan, 144, 144, nan, nan, nan, nan, nan, nan, nan, 134, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 107, nan, nan, nan, nan, nan, nan, nan, nan]
['MX000017004', 2010, 2, 'PRCP', nan, nan, nan, nan, nan, na

In [124]:
weather = open('weather.txt', 'r')
data_frame = pd.DataFrame(convert_data(weather, 2010, 5))
data_frame.columns = ["id_", "year", "month", "type"] + [i for i in range(1,32)]
data_frame.head()

,id_,year,month,type,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
0,MX000017004,2010,1,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.0,NaN
1,MX000017004,2010,1,TMIN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN
2,MX000017004,2010,1,PRCP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX000017004,2010,2,TMAX,NaN,273.0,241.0,NaN,NaN,NaN,...,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX000017004,2010,2,TMIN,NaN,144.0,144.0,NaN,NaN,NaN,...,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
# Ze względu na dużą liczbę wartości nieprawidłowych, dane można przetworzyć do postaci
# id_, year, month, day, type, value, co poprawia czytelność dostępnych danych.
data_frame = data_frame.melt(id_vars=["id_", "year", "month", "type"])
data_frame = data_frame[(pd.notna(data_frame["value"]))]
data_frame["day"] = data_frame["variable"]
data_frame = data_frame[["id_", "year", "month", "day", "type", "value"]]
data_frame.sort_values(["year", "month", "day"], inplace=True)

data_frame.to_csv("../AnalysisData/clean_weather.csv", index=False)
data_frame

,id_,year,month,day,type,value
435,MX000017004,2010,1,30,TMAX,278.0
436,MX000017004,2010,1,30,TMIN,145.0
18,MX000017004,2010,2,2,TMAX,273.0
19,MX000017004,2010,2,2,TMIN,144.0
33,MX000017004,2010,2,3,TMAX,241.0
34,MX000017004,2010,2,3,TMIN,144.0
153,MX000017004,2010,2,11,TMAX,297.0
154,MX000017004,2010,2,11,TMIN,134.0
333,MX000017004,2010,2,23,TMAX,299.0
334,MX000017004,2010,2,23,TMIN,107.0
